<a href="https://colab.research.google.com/github/aatika-hakim/AI/blob/main/research_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --upgrade --quiet langchain langchain-community langgraph langchain_google_genai pdfplumber langchain_core

In [7]:
!pip install --q pydantic

In [8]:
from google.colab import userdata
import os
import time
import json
import urllib3
import pdfplumber
from IPython.display import display, Markdown
# from langchain.document_loaders import PyPDFLoader
from langchain_core.messages import BaseMessage, SystemMessage, ToolMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from pydantic import BaseModel, Field
from typing import Optional, Annotated, Sequence, ClassVar, TypedDict

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')